In [ ]:
import pandas as pd

In [ ]:
def determine_grade(scores):
    if scores >= 93 and scores <= 100:
        return 'A'
    elif scores >= 90 and scores < 93:
        return 'A-'
    elif scores >= 87 and scores < 90:
        return 'B+'
    elif scores >= 83 and scores < 87:
        return 'B'
    elif scores >= 80 and scores < 83:
        return 'B-'
    elif scores >= 77 and scores < 80:
        return 'C+'
    elif scores >= 73 and scores < 77:
        return 'C'
    elif scores >= 70 and scores < 73:
        return 'C-'
    elif scores >= 67 and scores < 70:
        return 'D+'
    elif scores >= 63 and scores < 67:
        return 'D'
    elif scores >= 60 and scores < 63:
        return 'D-'
    else:
        return 'F'

In [ ]:
def meancalc(lst):
    nstu = 0
    tot = 0
    for i in lst:
        if i == 'Needs Grading':
            continue
        elif i == 'No Submission':
            continue
        elif i == 0:
            continue
        else:
            nstu+=1
            tot+=i
    return tot/nstu

# Some where in the data processing up here I should combine all the sections and assign a column with section name and assign a point total to each assignment

# When I do this, I need to figure out what to do with the honors work, maybe include a line that splits the honors info or adds a y/n honors column but then what do I do with non-honors students in the blank columns because some honors students didn't submit work so I would need an if for that sit.

So give a list of the things
loop over them to add the section number and honors information
then if there is an honors section

In [ ]:
sectList = ['01H','02','03','04','05']
dropColumns = ['Availability','Weighted Total','Total [Total','Safety Contract']

dataDict = {}
dataNames = []
for j in sectList:
    exec('data' + j + "= pd.read_csv('./exampledata/CEM153_'+j+'_Stripped.csv')")
    exec('data' + j + "['Section']=j")
    if 'H' in j:
        exec('data' + j + "['Honors']='Y'")
    else:
        exec('data' + j + "['Honors']='N'")
    dataDict['data'+j]=locals()['data'+j]
    temp = dataDict['data'+j]
    matching = [s for s in temp.columns if any(xs in s for xs in dropColumns)]
    temp = temp.drop(columns=matching)
    collist = []
    for k in temp.columns:
        end = k.find(']')
        if 'Attendance' in k:
            collist.append('Attendance')
        elif end != -1:
            collist.append(k[0:end+1])
        else:
            collist.append(k)
    temp.columns=collist
    temp = temp.set_index(['Username'])
    ### ADD THIS IN AS AN OPTION BUT I WANT IT IN FOR NOW ###
    deadColumnsBool = temp.isnull().values.all(axis=0)
    deadColumns = []
    for i in range(len(deadColumnsBool)):
        if deadColumnsBool[i] == True:
            ind = temp.columns[i]
            deadColumns.append(ind)
    temp = temp.drop(columns=deadColumns)
    temp = temp[:].fillna('No Submission')
    dataDict['data'+j]=temp
    dataNames.append('data'+j)

In [ ]:
dataNames

# Should I be using a dictionary instead of a pandas dataframe? 

# When I eventually load in all the sections for one class, I am going to need to compile a master list of students and their sections and stuff

# I think I want to make a schedule to mark the dates of the assignments so I can attach due dates to them, but that's wishful thinking

# When the files get imported and I bring in 5 files at onces, when I bring them in I should make sure there is a new column added with the section number.

# Right now I have everything organized by student but may need to change this to think about things by section. I feel like this is the same problem I had with my research in excel this summer so I gotta figure out how to transform or slice the data for the assignments section because I'll need info from each section for each assignment to be compiled. Idk if just assigning a section number as a column is the most efficient way to do this if I have loops on loops.

# I need an outside program to call my classes that can choose to do one student, students by section, or all students and same for assignments and all the combinations of the two. Something with flags/options and argparse

In [ ]:
### STUDENT ###
## If you give first/last name it will pull up a list of names to then select from
# Student Name (F&L), username, student id, Section Number**********<- need to add this
# Attendance: *gives percentage*
# Calculated Grade including Missing Assignments as 0's: *gives grade as %, letter*
# Calculated Grade excluding Missing Assignments: *gives grades as %, letter*
# Section Average: *gives section average grade as %, letter* <- going to have to think about this in terms of data transformation
# Course Average: *gives class average grade as %, letter*
# Missing Assignments: *gives list like: assignment 1, assignment 2, ...* (each item on separate line)

In [ ]:
### Included in class student ###
# Student Name                  as fname & lname
# username                      as uname
# student id                    as stuid
# section number                as sect
# honors status                 as honors
# attendance %                  as attendance
# allgrades %                   as allgradesp
# allgrades letter              as allgradesl
# onlygraded %                  as onlygradedp
# onlygraded letter             as onlygradedl
# list of missing work          as missingwork
# list of needs to be graded    as needsgrading

### To be included in outside loops ###
# List of potentially matching students or list of students & user names
# Section Average Grade
# Course Average Grade

In [ ]:
class student:
    def __init__(self,dataF,username):
        self.uname = username
        if username in dataF['Username']:
            self.fname = dataF.at[username,'First Name']
            self.lname = dataF.at[username,'Last Name']
            self.stuid = dataF.at[username,'Student ID']
            self.sect = dataF.at[username,'Section']
            self.honors = dataF.at[username,'Honors']
            self.attendance = dataF.at[username,'Attendance']
            self.studata = dataF.loc[[username]]
            
            dropColumns = []
            for j in dataF.columns:
                if 'Pts' in j:
                    continue
                else:
                    dropColumns.append(j)
            matching = [s for s in dataF.columns if any(xs in s for xs in dropColumns)]
            data2 = dataF.drop(columns=dropColumns)
            stot = 0
            allgrades = 0
            onlygraded = 0
            mwork = []
            ngrade = []
            for i in range(len(data2.columns)):
                colname = data2.columns[i]
                start = colname.find('[')
                end = colname.find(']')
                allgrades+=float(colname[start+12:end-6])
                if data2.at[self.uname,colname] == 'Needs Grading' or data2.at[self.uname,colname] == 'No Submission':
                    stot+=0
                    onlygraded+=0
                    if data2.at[self.uname,colname] == 'Needs Grading':
                        colname = colname[0:start]
                        ngrade.append(colname)
                    elif data2.at[self.uname,colname] == 'No Submission':
                        colname = colname[0:start]
                        mwork.append(colname)
                else:
                    onlygraded+=float(colname[start+12:end-6])
                    stot+=float(data2.at[self.uname,colname])
            self.allgradesp = (stot/allgrades)*100
            self.allgradesl = determine_grade(self.allgradesp)
            self.onlygradedp = (stot/onlygraded)*100
            self.onlygradedl = determine_grade(self.onlygradedp)
            if mwork == []:
                mwork = ['You have no work that is missing']
            if ngrade == []:
                ngrade = ['You have no work that needs grading']
            self.missingwork = mwork
            self.needsgrading = ngrade
        else:
            self.fname = ['There is no student with that name in the given data list']
            self.lname = ['There is no student with that name in the given data list']
            self.stuid = ['There is no student with that name in the given data list']
            self.sect = ['There is no student with that name in the given data list']
            self.honors = ['There is no student with that name in the given data list']
            self.attendance = ['There is no student with that name in the given data list']
            self.studata = ['There is no student with that name in the given data list']
            self.allgradesp = ['There is no student with that name in the given data list']
            self.allgradesl = ['There is no student with that name in the given data list']
            self.onlygradedp = ['There is no student with that name in the given data list']
            self.onlygradedl = ['There is no student with that name in the given data list']
            self.missingwork = ['There is no student with that name in the given data list']
            self.needsgrading = ['There is no student with that name in the given data list']

In [ ]:
dooku = student(dataDict['data03'],'cdooku')
jocasta = student(dataDict['data03'],'jnu')
tyranus = student(dataDict['data03'],'askywalker')
print('uname',tyranus.uname)
print('fname & lname',tyranus.fname,tyranus.lname)
print('stuid',tyranus.stuid)
print('sect',tyranus.sect)
print('honors',tyranus.honors)
print('attendance',tyranus.attendance)
print('studata',tyranus.studata)
print('allgradesp',tyranus.allgradesp)
print('allgradesl',tyranus.allgradesl)
print('onlygradedp',tyranus.onlygradedp)
print('onlygradedl',tyranus.onlygradedl)
print('missing work',tyranus.missingwork)
print('needs grading',tyranus.needsgrading)

In [ ]:
### ASSIGNMENT ###
## Gives a list of assignments to choose from if necessary 
# Assignment Name, points possible
# Calculated Avg for each section without 0's or missing assignmnets or 'Needs Grading'
# Calculated Avg for all sections ''
# Breakdown of who hasn't turned this in by section
# Breakdown of how many per section need to be graded
# Plot of section grades with course average as a line

In [ ]:
class assignment:
    
    def __init__(self,dataD,names,assignname):
        self.name = assignname
        
        start = self.name.find('[')
        end = self.name.find(']')
        self.points = float(self.name[start+12:end-6])
        
        adata = []
        sdata = []
        haves = []
        nots = []
        for j in names:
            temp = dataD[j]
            if assignname in temp.columns:
                sdata.append(temp[assignname].tolist())
                adata.extend(temp[assignname].tolist())
                haves.append(j)
            else:
                nots.append(j)
        if haves == []:
            haves.append('There are no assignments with this name in any of the sections')
            self.sectavgs = ['There are no assignments with this name in any of the sections']
            self.sectlets = ['There are no assignments with this name in any of the sections']
            self.courseavg = ['There are no assignments with this name in any of the sections']
            self.courselet = ['There are no assignments with this name in any of the sections']
        else:
            if len(sdata) > 1:
                sdatam = []
                sdatal = []
                for j in range(len(sdata)):
                    sdatam.append(meancalc(sdata[j]))
                    sdatal.append(determine_grade(sdatam[j]))
                self.sectavgs = sdatam
                self.sectlets = sdatal
                self.courseavg = meancalc(adata)
                self.courselet = determine_grade(self.courseavg)
            else:
                self.sectavgs = ['There is only one section with this assignment so sectavgs = courseavg']
                self.sectlets = ['There is only one section with this assignment so sectlets = courselet']
                self.courseavg = meancalc(adata)
                self.courselet = determine_grade(self.courseavg)
            if nots == []:
                nots.append('All sections have this assignment')
        self.coursedata = adata
        self.sectdata = sdata
        self.haves = haves
        self.nots = nots
        
        
        
        # Breakdown of who hasn't turned this in by section
        # Breakdown of how many per section need to be graded

# Need to remember Formal and Final are named different. Should loop through columnnames to be safe

In [ ]:
print(dataNames)

In [ ]:
lab1 = assignment(dataDict,dataNames,'L10 - Formal Report [Total Pts: 100 Score]') 
# print('name',lab1.name)
# print('points',lab1.points)
# print('haves',lab1.haves)
# print('nots',lab1.nots)
# print('course avg',lab1.courseavg)
# print('course let',lab1.courselet)
# print('sect avgs',lab1.sectavgs)
# print('sect lets',lab1.sectlets)

In [ ]:
print(dataNames)

# Should the course just be done in the outside code and not be a class because all the information I will need is stored in student and in assignment

In [ ]:
### COURSE ###
# Calculated Section Average
# Calculated Grade for All Sections
# Breakdown of Grades by assignment (section and class averages given)
# Compare sections with anova table
# Plots?